In [30]:
import os
current_directory = os.getcwd()
folder_name = os.path.basename(current_directory)
number = folder_name

In [31]:
CONFIG = {
    'data_main': 'C:/Users/Николай/PycharmProjects/CIBMTR/D.Data/main/',
    'test_path': 'test.csv',
    'train_path': 'train.csv',
    'folds_path': 'v1.csv', 
    'model_path': f"{number}_model.pkl",

    'DEVICE' : 'cuda',
    'SEED' : 42,
}

In [32]:
from catboost import CatBoostRegressor
import joblib
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
import joblib 
from metric import score_
from lifelines import KaplanMeierFitter

In [33]:
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    return y

In [34]:
# Load and prepare data
train = pd.read_csv(f"{CONFIG['data_main']}{CONFIG['train_path']}")
train = train.fillna('-1')

for col in ['donor_age', 'age_at_hct']:
    train[col] = train[col].astype(int)
    
train["y"] = transform_survival_probability(train, time_col='efs_time', event_col='efs')

cat_columns = [col for col in train.columns if col not in ['efs', 'efs_time', 'y']]
train[cat_columns] = train[cat_columns].astype(str)

In [35]:
test = pd.read_csv(f"{CONFIG['data_main']}test.csv")
test = test.fillna('-1')
for col in ['donor_age', 'age_at_hct']:
    test[col] = test[col].astype(int)
test = test.astype(str)

In [36]:
final_cat = joblib.load(CONFIG['model_path'])

In [42]:
preds_test = final_cat.predict(test.drop(columns=['ID', 'race_group']))

In [45]:
y_pred = test[['ID']].copy(deep=True)
y_pred["prediction"] = preds_test
y_pred.to_csv('submission.csv', index=False)